# Debug MM GAN


In [1]:

import numpy as np
import sys
sys.path.append('/home/tl3/repos/project-rcGAN/jess_version/rcGAN/')

from torch.utils.data import DataLoader
import pytorch_lightning as pl
from models.lightning.rcGAN import rcGAN
from models.archs.mri.generator import UNetModel
from models.archs.mri.discriminator import DiscriminatorModel

import yaml
import json
import types
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


/home/tl3/repos/project-rcGAN/jess_version/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [2]:

device


device(type='cuda')

In [3]:

def load_object(dct):
    return types.SimpleNamespace(**dct)

with open('/home/tl3/repos/project-rcGAN/jess_version/rcGAN/configs/mass_map.yml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg = json.loads(json.dumps(cfg), object_hook=load_object)


In [4]:


model = rcGAN(cfg, exp_name='debug_model', num_gpus=0)


In [5]:
cfg.im_size

1024

In [6]:
from data.lightning.MassMappingDataModule import MMDataTransform
from data.datasets.MM_data import MassMappingDataset_Test, MassMappingDataset_Train, MassMappingDataset_Val


data_dir = '/share/gpu0/tl3/mass_map_dataset/kappa_debug/'


mm_data_transform = MMDataTransform(args=None)
mm_dataset_train = MassMappingDataset_Train(data_dir=data_dir, transform=mm_data_transform)
new_data = mm_dataset_train.__getitem__(0)

# Input batch
y, x, mean, std = new_data


x = x[None,:,:,:].to(device)
y = y[None,:,:,:].to(device)


In [7]:
y.dtype

torch.float32

In [8]:
x.dtype

torch.float32

In [10]:


def get_noise(num_vectors, resolution=1024):
    z = torch.randn(num_vectors, 2, resolution, resolution, device=device)
    return z

num_vectors = y.size(0)
noise = get_noise(num_vectors)

input_data = torch.cat([y, noise], dim=1)


In [11]:
input_data.dtype

torch.float32

In [12]:
torch.cuda.init()
torch.set_float32_matmul_precision('medium')


In [14]:

# with torch.cuda.device(0):

generator = UNetModel(
    in_chans=4,
    out_chans=2,
)
generator.to(device)



UNetModel(
  (down_sample_layers): ModuleList(
    (0): ConvDownBlock(
      (conv_1): Conv2d(4, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (res): ResidualBlock(
        (conv_block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): PReLU(num_parameters=1)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): PReLU(num_parameters=1)
        )
        (conv_1x1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
      )
      (conv_3): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): PReLU(num_parameters=1)
    )
    (1): ConvDownBlock(
      (c

In [15]:

sample = generator(input_data)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 15.78 GiB total capacity; 14.21 GiB already allocated; 351.50 MiB free; 14.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF